<a href="https://colab.research.google.com/github/josesg998/competencia_properati/blob/main/TP_Individual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

In [11]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Lectura de datos

In [ ]:
#DIR = "/content/drive/MyDrive/Data Mining Properati/"

#df_train = pd.read_feather(DIR + 'properati_entrenamiento.ftr').set_index('id')
#df_test = pd.read_feather(DIR + "properati_a_predecir.ftr").set_index('id')

In [14]:
#df_train = pd.read_csv(DIR + "properati_entrenamiento.zip", index_col="id")  
#df_train.head()

# df_test = pd.read_csv(DIR + "properati_a_predecir.zip", index_col="id")  
#df_test.head()

# df_train = pd.read_csv('bases/properati_entrenamiento.csv',index_col='id')
# df_test = pd.read_csv('bases/properati_a_predecir.csv',index_col='id')
# df_test.head()

# df_train.reset_index(drop=False).to_feather('bases/properati_entrenamiento.ftr')
# df_test.reset_index(drop=False).to_feather('bases/properati_a_predecir.ftr')

In [2]:
df_train = pd.read_feather('bases/properati_entrenamiento.ftr').set_index('id')
df_test = pd.read_feather('bases/properati_a_predecir.ftr').set_index('id')
df_train.head(3)

,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
id,,,,,,,,,,,,,,,,,,,,,
1,Propiedad,2019-07-07,2019-07-10,2019-07-07,NaN,NaN,Argentina,Buenos Aires Interior,Urquiza,Urquiza,...,NaN,NaN,NaN,3800.0,ARS,Mensual,COCHERA EN ALQUILER,Xintel (SLC-SI1-560) Alquiler de Cochera en CA...,Cochera,Alquiler
2,Propiedad,2019-07-07,9999-12-31,2019-07-07,-64.184812,-31.421307,Argentina,Córdoba,Córdoba,None,...,NaN,12.0,12.0,16000.0,USD,Mensual,Cochera venta,VENDO COCHERA- EXCELENTE UBICACIÓN! Calle Buen...,Cochera,Venta
3,Propiedad,2019-07-07,2019-07-09,2019-07-07,-57.965880,-34.902568,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,La Plata,...,NaN,12.0,NaN,16000.0,USD,None,Cochera cubierta con portón automatizado en ve...,Cochera Cubierta en venta ubicada en calle 7 e...,Cochera,Venta


# Entendimiento (magia parte 1 -- AID)

In [3]:
df = pd.concat((df_train,df_test))

In [4]:
df_train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ad_type,994000,1,Propiedad,994000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
start_date,994000,346,2020-03-21,13171,NaN,NaN,NaN,NaN,NaN,NaN,NaN
end_date,994000,450,9999-12-31,190441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
created_on,994000,346,2020-03-21,13171,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lat,840826.0,NaN,NaN,NaN,-59.264786,2.704534,-180.0,-58.754528,-58.44429,-58.069144,-35.045025
lon,841760.0,NaN,NaN,NaN,-34.793299,3.062879,-54.842464,-34.920924,-34.610161,-34.460719,85.051129
l1,994000,4,Argentina,975230,NaN,NaN,NaN,NaN,NaN,NaN,NaN
l2,994000,42,Capital Federal,251843,NaN,NaN,NaN,NaN,NaN,NaN,NaN
l3,936714,1262,Mar del Plata,158074,NaN,NaN,NaN,NaN,NaN,NaN,NaN
l4,225232,976,Nordelta,17310,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 994000 entries, 1 to 1000000
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ad_type          994000 non-null  object 
 1   start_date       994000 non-null  object 
 2   end_date         994000 non-null  object 
 3   created_on       994000 non-null  object 
 4   lat              840826 non-null  float64
 5   lon              841760 non-null  float64
 6   l1               994000 non-null  object 
 7   l2               994000 non-null  object 
 8   l3               936714 non-null  object 
 9   l4               225232 non-null  object 
 10  l5               4711 non-null    object 
 11  l6               0 non-null       float64
 12  rooms            504328 non-null  float64
 13  bedrooms         391380 non-null  float64
 14  bathrooms        774749 non-null  float64
 15  surface_total    451104 non-null  float64
 16  surface_covered  439770 non-null  flo

In [7]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ad_type,1000000,1,Propiedad,1000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
start_date,1000000,346,2020-03-21,13194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
end_date,1000000,450,9999-12-31,191629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
created_on,1000000,346,2020-03-21,13194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lat,846370.0,NaN,NaN,NaN,-59.259418,2.696735,-180.0,-58.750954,-58.443991,-58.087463,-35.045025
lon,847305.0,NaN,NaN,NaN,-34.792021,3.052942,-54.842464,-34.919763,-34.609836,-34.464277,85.051129
l1,1000000,4,Argentina,981230,NaN,NaN,NaN,NaN,NaN,NaN,NaN
l2,1000000,42,Capital Federal,257546,NaN,NaN,NaN,NaN,NaN,NaN,NaN
l3,942503,1262,Mar del Plata,158074,NaN,NaN,NaN,NaN,NaN,NaN,NaN
l4,225691,976,Nordelta,17310,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 1 to 757314
Data columns (total 24 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   ad_type          1000000 non-null  object 
 1   start_date       1000000 non-null  object 
 2   end_date         1000000 non-null  object 
 3   created_on       1000000 non-null  object 
 4   lat              846370 non-null   float64
 5   lon              847305 non-null   float64
 6   l1               1000000 non-null  object 
 7   l2               1000000 non-null  object 
 8   l3               942503 non-null   object 
 9   l4               225691 non-null   object 
 10  l5               4711 non-null     object 
 11  l6               0 non-null        float64
 12  rooms            510328 non-null   float64
 13  bedrooms         396315 non-null   float64
 14  bathrooms        780531 non-null   float64
 15  surface_total    455880 non-null   float64
 16  surface_covered  44

# Limpieza y transformación de datos (magia parte 2 -- MD)

In [ ]:
## esto tiene que hacerse mejor
df_train.fillna(0, inplace=True, downcast= "infer")
df_test.fillna(0, inplace=True, downcast= "infer") 

(11100, 24)

In [ ]:
# Funcion de limpieza (usada en la siguiente sección) que usa y_train
def limpiar_fold(X_train, y_train, X_test):
    ## TODO: limpiar los datos

    return (X_train, y_train, X_test)

# Entrenamiento del modelo

## Territorio de Aprendizaje Automático
☢☢☢☢☢☢ Se mira y no se toca ☢☢☢☢☢☢☢

In [ ]:
# Datos para probar
df_train = df_train.select_dtypes(include=['float64' ,'float32','int64', 'int32', 'int16', 'int8', 'bool'])

X = df_train[df_train.columns.drop('price')]
y = df_train['price']

In [ ]:
# Creamos el modelo
reg = sk.ensemble.RandomForestRegressor(n_estimators=500, max_depth=5, n_jobs=-1, random_state=42)

# Partimos en entrenamiento+prueba y validación
X_train_test, X_val, y_train_test, y_val = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

scores_train = []
scores_test = []
# Validación cruzada, 10 folds, shuffle antes
kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
for fold, (train_index, test_index) in enumerate(kf.split(X_train_test, y_train_test)):
    X_train, X_test, y_train, y_test = X_train_test.iloc[train_index], X_train_test.iloc[test_index], y_train_test.iloc[train_index], y_train_test.iloc[test_index]

    X_train, y_train, X_test = limpiar_fold(X_train, y_train, X_test)

    # Entrenamos el modelo
    reg.fit(X_train, y_train)

    # Predecimos en train
    y_pred = reg.predict(X_train)

    # Medimos la performance de la predicción en test
    score_train = sk.metrics.mean_squared_error(y_train, y_pred)
    scores_train.append(score_train)

    # Predecimos en test
    y_pred = reg.predict(X_test)    

    # Medimos la performance de la predicción en test
    score_test = sk.metrics.mean_squared_error(y_test, y_pred)
    scores_test.append(score_test)
    
    print(f"{fold=}, {score_train=} {score_test=}")
    

print(f"Train scores mean={pd.Series(scores_train).mean()}, std={pd.Series(scores_train).std()}")
print(f"Test scores mean={pd.Series(scores_test).mean()}, std={pd.Series(scores_test).std()}")

NameError: name 'limpiar_fold' is not defined

# Predicción para kaggle


In [ ]:
## Datos a predecir 
X = df_train[df_train.columns.drop('price')]
y = df_train['price']
X_prueba = df_test[df_train.columns.drop('price')] # cuidado:

# Entrenamos el modelo con todos los datos
reg.fit(X, y)

# Predecimos
df_test['price'] = reg.predict(X_prueba)

# Grabamos
df_test['price'].to_csv('solucion.csv', index=True)

In [ ]:
pd.DataFrame(zip(df_train.columns.drop('price'), reg.feature_importances_), columns=["column", "feature_importance"]).sort_values(by="feature_importance", ascending=False)

,column,feature_importance
1,lon,0.284658
0,lat,0.212267
4,rooms,0.141806
8,surface_covered,0.125101
5,bedrooms,0.095744
7,surface_total,0.073814
6,bathrooms,0.066611
2,l5,0.000000
3,l6,0.000000
